In [ ]:
# import and setup environment
import twitter
import pandas as pd
import numpy as np
import random
from bokeh.plotting import figure, output_notebook,show
output_notebook()
%matplotlib inline

import requests
import json
import matplotlib.pyplot as plt

#enter your own keys to test Mine will be hidden 
CONSUMER_KEY = 'hidden'
CONSUMER_SECRET ='hidden'
OAUTH_TOKEN = 'hidden'
OAUTH_TOKEN_SECRET = 'hidden'

In [ ]:
auth = twitter.oauth.OAuth(OAUTH_TOKEN, OAUTH_TOKEN_SECRET,
                           CONSUMER_KEY, CONSUMER_SECRET)

twitter_api = twitter.Twitter(auth=auth)



#  Q: Most trending topics in USA Regions and CA Regions

In [ ]:
#trending locations for twitter
trending_loc = twitter_api.trends.available()
trending_loc
loc_df = pd.DataFrame(trending_loc)
loc_df.head()

In [ ]:
loc_df[loc_df['parentid']==1]

In [ ]:
#grab woeid for canada and usa. 
#covnert to  list to call trend api for each location in usa and canada respectively 
us_woeid_list = loc_df[loc_df['countryCode']=='US'][loc_df['parentid'] != 1]['woeid'].tolist()
ca_woeid_list = loc_df[loc_df['countryCode']=='CA'][loc_df['parentid'] != 1]['woeid'].tolist()
print us_woeid_list[:5]
print ca_woeid_list[:5]

In [ ]:
#since twitter trends api sets a limit to the calls we can make and does not allow chaining of woeids.
#randomly select 5 from us and 5 from ca
us_woeid_subset = random.sample(set(us_woeid_list), 5)
ca_woeid_subset = random.sample(set(ca_woeid_list),5)
print us_woeid_subset
print ca_woeid_subset

In [ ]:
usa_trends = []
ca_trends = []

for i, _val in enumerate(us_woeid_subset):
    usa_trends.append(twitter_api.trends.place(_id=us_woeid_subset[i]))
    ca_trends.append(twitter_api.trends.place(_id=ca_woeid_subset[i]))
    

In [ ]:
from collections import defaultdict
usa_queries = defaultdict(int)
ca_queries = defaultdict(int)
agg_queries = defaultdict(int)

for array in usa_trends:
    for el in array[0]['trends']:
        usa_queries[el['name']] +=1
        agg_queries[el['name']] +=1

for array in ca_trends:
    for el in array[0]['trends']:
        ca_queries[el['name']] +=1
        agg_queries[el['name']] +=1

print usa_queries
print ca_queries


In [ ]:
usa_df = pd.DataFrame(usa_queries.items(),columns=['query','counts'])
ca_df = pd.DataFrame(ca_queries.items(),columns=['query','counts'])
agg_df = pd.DataFrame(agg_queries.items(),columns=['query','counts'])
print usa_df.head()
print ca_df.head()
print agg_df.head()

In [ ]:
#prepare data
usa_df = usa_df.set_index(usa_df['query'])
usa_df = usa_df.drop('query', 1)
ca_df = ca_df.set_index(ca_df['query'])
ca_df = ca_df.drop('query',1)
agg_df = agg_df.set_index(agg_df['query'])
agg_df = agg_df.drop('query',1)


In [ ]:
ca_df

In [ ]:
ca_df.oc

In [ ]:
from bokeh.charts import Bar

show(Bar(usa_df, title ='USA'))
show(Bar(ca_df, title="CA"))

In [ ]:
show(Bar(agg_df[agg_df.counts > 1], title="US + Canada"))

# Conclusion

Out of our random sample of 5 regions in Canada and 5 regions in USA looks like Fallout 4 are the shared trends.
Results will change on each run due to random selection process and constantly changing trends on twitter